# Custom Jobs with custom container in Vertex AI

In [28]:
# pip install or import packages
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    ! pip install google-cloud-service-usage -q
try:
    import google.cloud.artifactregistry_v1 
except ImportError:
    print('You need to pip install google-cloud-artifact-registry')
    ! pip install google-cloud-artifact-registry -q
try:
    import google.cloud.devtools.cloudbuild
except ImportError:
    print("You need to pip install google-cloud-build")
    !pip install google-cloud-build
from google.cloud import aiplatform
from datetime import datetime
from IPython.display import Markdown as md
from google.cloud import service_usage_v1
from google.cloud.devtools import cloudbuild_v1
from google.cloud import artifactregistry_v1
from google.cloud import storage
from google.cloud import bigquery
from google.protobuf.struct_pb2 import Value
import json
import numpy as np
import pandas as pd

# Set up environment 

In [29]:
project = !gcloud config get-value project

In [30]:
PROJECT_ID = project[0]
REGION = 'australia-southeast1'
EXPERIMENT = '04'
SERIES = '04'

In [31]:
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'media_campaign_cost'
BQ_TABLE = 'mcc_train'

In [216]:
BASE_IMAGE = 'us-docker.pkg.dev/deeplearning-platform-release/gcr.io/base-cu113.py310'
#DEPLOY_IMAGE = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest'

DEPLOY_IMAGE = 'australia-southeast1-docker.pkg.dev/my-project-media-campaign-cost/my-project-media-campaign-cost4/04_trainer:latest'
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

In [33]:
TIMESTAMP =datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}"
DIR = f"temp/{EXPERIMENT}"

In [34]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)'
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]

In [35]:
!rm -rf {DIR}
!mkdir -p {DIR}

In [220]:
# Experiment Tracking
FRAMEWORK = 'sklearn'
TASK = 'regression'
MODEL_TYPE = 'ensemble'
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-{FRAMEWORK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

In [37]:
# create a list of clients 
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project=PROJECT_ID)
gcs = storage.Client(project=PROJECT_ID)
su_client = service_usage_v1.ServiceUsageClient()
ar_client = artifactregistry_v1.ArtifactRegistryClient()
cb_client = cloudbuild_v1.CloudBuildClient()

# Set up Vertex AI experiment

In [38]:
aiplatform.init(experiment=EXPERIMENT_NAME)

# Training 

In [177]:
# have a look at the training python script
script_path = './trainer/ensemble.py'
with open(script_path, 'r') as file:
    data = file.read()
md(f"```python\n\n{data}\n```")

```python

#import packages
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, TargetEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import VotingRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import pandas as pd
import numpy as np
import pickle
from google.cloud import bigquery 
from google.cloud import aiplatform
from google.cloud import storage
import hypertune
import argparse
import os
import sys
# local parameters
parser =  argparse.ArgumentParser()
parser.add_argument('--project_id',dest ='project_id',type=str)
parser.add_argument('--bq_project',dest ='bq_project', type=str)
parser.add_argument('--bq_dataset',dest ='bq_dataset', type=str)
parser.add_argument('--bq_table', dest ='bq_table',type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--experiment', dest='experiment',type=str)
parser.add_argument('--series', dest='series', type=str)
parser.add_argument('--experiment_name',dest='experiment_name',type=str)
parser.add_argument('--run_name', dest = 'run_name', type = str)
# xgb&lightgbm
parser.add_argument('--x_seed', dest = 'x_seed', type = int)
parser.add_argument('--l_seed', dest = 'l_seed', type = int)
parser.add_argument('--x_objective', dest='x_objective', type = str)
parser.add_argument('--l_objective', dest='l_objective', type = str)
parser.add_argument('--x_max_depth', dest='x_max_depth',type=int)
parser.add_argument('--l_max_depth', dest='l_max_depth',type=int)
#xgb&cat
parser.add_argument('--x_grow_policy', dest='x_grow_policy', type = str)
parser.add_argument('--c_grow_policy', dest='c_grow_policy', type = str)
#lightgbm & cat
parser.add_argument('--l_learning_rate', dest='l_learning_rate', type = float)
parser.add_argument('--c_learning_rate', dest='c_learning_rate', type = float)
# xgb 
parser.add_argument('--eval_metric', dest='eval_metric', type = str)
parser.add_argument('--tree_method', dest='tree_method', type = str)
parser.add_argument('--eta', dest="eta", type=float)
parser.add_argument('--alpha', dest='alpha', type=float)
#lighgbm
parser.add_argument('--metric', dest='metric', type = str)
parser.add_argument('--feature_fraction', dest='feature_fraction', type=float)
parser.add_argument('--bagging_fraction', dest='bagging_fraction', type=float)
parser.add_argument('--verbose', dest='verbose', type=int)
parser.add_argument('--num_leaves', dest='num_leaves', type = int)
parser.add_argument('--lambda_l1', dest='lambda_l1', type=float)
#catboost
parser.add_argument('--random_seed', dest = 'random_seed', type = int)
parser.add_argument('--loss_function', dest='loss_function', type=str)
parser.add_argument('--depth', dest='depth', type = int)
parser.add_argument('--iterations', dest='iterations',type=int)
parser.add_argument('--l2_leaf_reg', dest='l2_leaf_reg', type=float)
args = parser.parse_args()
# create clients
bq =  bigquery.Client(project=args.project_id)
aiplatform.init(project = args.project_id, location = args.region)
#vertex ai experiment 
if args.run_name in [run.name for run in aiplatform.ExperimentRun.list(experiment = args.experiment_name)]:
    expRun = aiplatform.ExperimentRun(run_name = args.run_name, experiment = args.experiment_name)
else:
    expRun = aiplatform.ExperimentRun.create(run_name = args.run_name, experiment = args.experiment_name)
expRun.log_params({'experiment':args.experiment, 'series':args.series, 'project_id': args.project_id})
expRun.log_params({'data_source':f"bq://{args.bq_project}.{args.bq_dataset}.{args.bq_table}"})
# extract data from bigquery table
# drop recyclable_packge and low_fat due to the feature importance we found in previous hyperparameter tuning job.
# feature engineering: creat home_children_ratio, drop prepared_food since high-correlation to salad_bar
query = f"SELECT store_sales_in_millions_, unit_sales_in_millions_, total_children-num_children_at_home AS independent_children,gross_weight, avg_cars_at_home_approx__1, units_per_case, store_sqft, coffee_bar+video_store+salad_bar+florist AS store_score, cost FROM `{args.bq_project}.{args.bq_dataset}.{args.bq_table}`"
df = bq.query(query).to_dataframe()
# extract target variable and explainatory variables
y = df['cost']
x = df.drop('cost', axis="columns")
# data preprocessing 
num_attrib = ["independent_children","avg_cars_at_home_approx__1", "store_sqft", "store_score"]
preprocess =  ColumnTransformer([("numerical", TargetEncoder(), num_attrib)],remainder='passthrough')
x_processed = preprocess.fit_transform(x,y)
# split data into train and test data sets
x_train,x_test,y_train,y_test =  train_test_split(x_processed, y, train_size = 0.8, test_size = 0.2, random_state=50)
#xgb_model parameters
xgb_params = {
    'seed': args.x_seed,
    'objective': args.x_objective,
    'eval_metric': args.eval_metric,
    'eta': args.eta,
    'max_depth': args.x_max_depth,
    'alpha': args.alpha,
    'tree_method': args.tree_method,
    'grow_policy': args.x_grow_policy
}
#lgbm_model parameters
lgbm_params = {
    'seed': args.l_seed,
    'objective': args.l_objective,
    'metric': args.metric,
    'learning_rate': args.l_learning_rate,
    'max_depth': args.l_max_depth,
    'lambda_l1': args.lambda_l1,
    'num_leaves': args.num_leaves,
    'bagging_fraction': args.bagging_fraction,
    'feature_fraction': args.feature_fraction,
    'verbose': args.verbose
}
#catboost_model parameters
catboost_params = {
    'random_seed': args.random_seed,
    'learning_rate': args.c_learning_rate,
    'iterations': args.iterations,
    'l2_leaf_reg': args.l2_leaf_reg,
    'depth': args.depth,
    'loss_function': args.loss_function,
    'grow_policy': args.c_grow_policy,
}
# build model 
xgb_model = XGBRegressor(**xgb_params)
lgbm_model = LGBMRegressor(**lgbm_params)
cat_model = CatBoostRegressor(**catboost_params)
ensemble_model = VotingRegressor([("xgb", xgb_model),("lgbm",lgbm_model),("catboost",cat_model)])
# train model 
model_ensemble = ensemble_model.fit(x_train, y_train)
# evaluate using msle metric
y_train_pred = model_ensemble.predict(x_train)
train_msle = mean_squared_log_error(y_train_pred, y_train)
y_test_pred =  model_ensemble.predict(x_test)
test_msle = mean_squared_log_error(y_test_pred, y_test)
expRun.log_params({"train_msle": train_msle, "test_msle": test_msle})
# save the model
file_name = 'ensemble_model.pkl'
# Use predefined environment variable to establish model directionary
model_directory = os.environ['AIP_MODEL_DIR']
storage_path = f'/gcs/{model_directory[5:]}'+file_name
os.makedirs(os.path.dirname(storage_path), exist_ok=True)
# output the model save files directly to GCS destination
with open (storage_path,'wb') as f:
    pickle.dump(model_ensemble,f)
expRun.log_params({'model.save': storage_path})
expRun.end_run()

```

# Create a custom container


In [178]:
# get the source path in google cloud storage
bucket = gcs.lookup_bucket(BUCKET)
if not bucket:
    gcs.bucket(BUCKET).create(location=REGION)
SOURCEPATH = f'{SERIES}/{EXPERIMENT}/training' 

In [179]:
# upload training code to google cloud storage
blob = storage.Blob(f'{SOURCEPATH}/{EXPERIMENT}_trainer/train.py', bucket=gcs.bucket(BUCKET))
blob.upload_from_filename(script_path)

In [180]:
#create requirement file for docker file
requirements = f""" google-cloud-aiplatform
protobuf
db-dtypes>=1.0.0
google-auth>=2.6.0
google-cloud-bigquery>=3.0.1
cloudml-hypertune
xgboost
lightgbm
catboost
"""
blob = storage.Blob(f'{SOURCEPATH}/requirements.txt', bucket=gcs.bucket(BUCKET))
blob.upload_from_string(requirements)

In [181]:
# create the Dockerfile and upload to gcs
dockerfile = f'''
FROM {BASE_IMAGE}
WORKDIR /training
# copy requirements and install them
COPY requirements.txt ./
RUN pip install --no-cache-dir --upgrade pip \
  && pip install --no-cache-dir -r requirements.txt
## Copies the trainer code to the docker image
COPY {EXPERIMENT}_trainer/* ./{EXPERIMENT}_trainer/
## Sets up the entry point to invoke the trainer
ENTRYPOINT ["python", "-m", "{EXPERIMENT}_trainer.train"]
'''
blob = storage.Blob(f'{SOURCEPATH}/Dockerfile', bucket=gcs.bucket(BUCKET))
blob.upload_from_string(dockerfile)

In [182]:
## create docker image repository 
docker_repo = None
for repo in ar_client.list_repositories(parent = f"projects/{PROJECT_ID}/locations/{REGION}"):
    if repo.labels['experiment']==EXPERIMENT:
        docker_repo = repo
        print(f'Retrieved existing repo:{docker_repo.name}')
if not docker_repo:
    operation = ar_client.create_repository(
    request = artifactregistry_v1.CreateRepositoryRequest(
        parent = f'projects/{PROJECT_ID}/locations/{REGION}',
        repository_id =f'{PROJECT_ID}4',
        repository = artifactregistry_v1.Repository(
            description=f'A repository for the {EXPERIMENT} experiment',
            name = f'{PROJECT_ID}',
            format_ = artifactregistry_v1.Repository.Format.DOCKER,
            labels = {'series':SERIES, 'experiment':EXPERIMENT}
    )
    )
    )
    print ("Creating Repository ...")
    docker_repo = operation.result()
    print(f'Complete creating repo: {docker_repo.name}')

Retrieved existing repo:projects/my-project-media-campaign-cost/locations/australia-southeast1/repositories/my-project-media-campaign-cost4


In [183]:
docker_repo.name, docker_repo.format_.name

('projects/my-project-media-campaign-cost/locations/australia-southeast1/repositories/my-project-media-campaign-cost4',
 'DOCKER')

In [184]:
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{docker_repo.name.split('/')[-1]}"

In [185]:
# setup the build config with empty list of steps - these will be added sequentially
build = cloudbuild_v1.Build(
    steps = []
)
# retrieve the source
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/gsutil',
        'args': ['cp', '-r', f'gs://{PROJECT_ID}/{SOURCEPATH}/*','/workspace']
    }
)
# docker build
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/docker',
        'args': ['build', '-t', f'{REPOSITORY}/{EXPERIMENT}_trainer','/workspace']
    }    
)

# docker push
build.images = [f"{REPOSITORY}/{EXPERIMENT}_trainer"]



operation = cb_client.create_build(
    project_id = PROJECT_ID,
    build = build
)

response = operation.result()
response.status, response.artifacts

(<Status.SUCCESS: 3>,
 images: "australia-southeast1-docker.pkg.dev/my-project-media-campaign-cost/my-project-media-campaign-cost4/04_trainer")

In [186]:
#xgb&lgbm
X_SEED = '50'
L_SEED = '50'
X_OBJECTIVE = 'reg:squaredlogerror'
L_OBJECTIVE = 'regression'
X_MAX_DEPTH = '10'
L_MAX_DEPTH = '20'
#xgb&catboot
X_GROW_POLICY = 'lossguide'
C_GROW_POLICY = 'Lossguide'
#lgbm&cat
L_LEARNING_RATE = '0.1726402706922235'
C_LEARNING_RATE='0.30403279991806864'
#xgb
EVAL_METRIC = 'rmsle'
TREE_METHOD = 'hist'
ETA = '0.6877740759969978'
ALPHA = '0.054789458485290723'
#lgbm
METRIC = 'mse'
FEATURE_FRACTION = '0.9'
BAGGING_FRACTION = '0.7'
VERBOSE='-1' 
NUMBER_LEAVES = '40'
LAMBDA_L1 = '2.0'
#cat
RANDOM_SEED = '50'
LOSS_FUNCTION = 'RMSE'
DEPTH = '10'
ITERATIONS = '100'
L2_LEAF_REG = '2.0'

In [187]:
CMDARGS = [
    "--project_id=" + PROJECT_ID,
    "--bq_project=" + BQ_PROJECT,
    "--bq_dataset=" + BQ_DATASET,
    "--bq_table=" + BQ_TABLE,
    "--region=" + REGION,
    "--experiment=" + EXPERIMENT,
    "--series=" + SERIES,
    "--experiment_name=" + EXPERIMENT_NAME,
    "--run_name=" + RUN_NAME,
    "--x_seed=" + X_SEED,
    "--l_seed=" + L_SEED,
    "--x_objective=" + X_OBJECTIVE,
    "--l_objective="+ L_OBJECTIVE,
    "--x_max_depth=" + X_MAX_DEPTH,
    "--l_max_depth="+ L_MAX_DEPTH,
    "--x_grow_policy=" + X_GROW_POLICY,
    "--c_grow_policy=" + C_GROW_POLICY,
    "--l_learning_rate=" + L_LEARNING_RATE,
    "--c_learning_rate=" + C_LEARNING_RATE,
    "--eval_metric="+ EVAL_METRIC,
    "--tree_method="+ TREE_METHOD,
    "--eta=" + ETA,
    "--alpha=" + ALPHA,
    "--metric=" + METRIC,
    "--feature_fraction=" + FEATURE_FRACTION,
    "--bagging_fraction=" + BAGGING_FRACTION,
    "--verbose=" + VERBOSE,
    "--num_leaves="+ NUMBER_LEAVES,
    "--lambda_l1="+LAMBDA_L1,
    "--random_seed=" + RANDOM_SEED,
    "--loss_function=" + LOSS_FUNCTION,
    "--depth=" + DEPTH, 
    "--iterations="+ITERATIONS,
    "--l2_leaf_reg="+L2_LEAF_REG,
]


In [188]:
CMDARGS

['--project_id=my-project-media-campaign-cost',
 '--bq_project=my-project-media-campaign-cost',
 '--bq_dataset=media_campaign_cost',
 '--bq_table=mcc_train',
 '--region=australia-southeast1',
 '--experiment=04',
 '--series=04',
 '--experiment_name=experiment-04-04-sklearn-regression-ensemble',
 '--run_name=run-20231229122112',
 '--x_seed=50',
 '--l_seed=50',
 '--x_objective=reg:squaredlogerror',
 '--l_objective=regression',
 '--x_max_depth=10',
 '--l_max_depth=20',
 '--x_grow_policy=lossguide',
 '--c_grow_policy=Lossguide',
 '--l_learning_rate=0.1726402706922235',
 '--c_learning_rate=0.30403279991806864',
 '--eval_metric=rmsle',
 '--tree_method=hist',
 '--eta=0.6877740759969978',
 '--alpha=0.054789458485290723',
 '--metric=mse',
 '--feature_fraction=0.9',
 '--bagging_fraction=0.7',
 '--verbose=-1',
 '--num_leaves=40',
 '--lambda_l1=2.0',
 '--random_seed=50',
 '--loss_function=RMSE',
 '--depth=10',
 '--iterations=100',
 '--l2_leaf_reg=2.0']

In [189]:
MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}

WORKER_POOL_SPEC = [
    {
        "replica_count":1,
        "machine_spec":MACHINE_SPEC,
        "container_spec":{
            "image_uri": f"{REPOSITORY}/{EXPERIMENT}_trainer",
            "args":CMDARGS,
        },
    }
]

In [190]:
customJob = aiplatform.CustomJob(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    worker_pool_specs = WORKER_POOL_SPEC,
    base_output_dir = f"{URI}/models/{TIMESTAMP}",
    staging_bucket = f"{URI}/models/{TIMESTAMP}",
    labels = {'series': f'{SERIES}', 'experiment':f'{EXPERIMENT}','experiment_name':f'{EXPERIMENT_NAME}'}
)

In [191]:
customJob.run(
    service_account = SERVICE_ACCOUNT
)

Creating CustomJob
CustomJob created. Resource name: projects/683212519680/locations/australia-southeast1/customJobs/1231464018225397760
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/683212519680/locations/australia-southeast1/customJobs/1231464018225397760')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/australia-southeast1/training/1231464018225397760?project=683212519680
CustomJob projects/683212519680/locations/australia-southeast1/customJobs/1231464018225397760 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/683212519680/locations/australia-southeast1/customJobs/1231464018225397760 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/683212519680/locations/australia-southeast1/customJobs/1231464018225397760 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/683212519680/locations/australia-southeast1/customJobs/1231464018225397760 current state:
JobState.JOB_STATE_PENDING
CustomJob

NameError: name 'tuningJob' is not defined

In [192]:
customJob.resource_name, customJob.display_name

('projects/683212519680/locations/australia-southeast1/customJobs/1231464018225397760',
 '04_04_20231229122112')

In [193]:
job_link = f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/training/{customJob.resource_name.split('/')[-1]}/cpu?cloudshell=false&project={PROJECT_ID}"

print(f'Review the Custom Job here:\n{job_link}')

Review the Custom Job here:
https://console.cloud.google.com/vertex-ai/locations/australia-southeast1/training/1231464018225397760/cpu?cloudshell=false&project=my-project-media-campaign-cost


# Upload the model to repository

In [218]:
modelmatch = aiplatform.Model.list(filter = f'display_name={SERIES}_{EXPERIMENT} AND labels.series={SERIES} AND labels.experiment={EXPERIMENT}')

upload_model = True
if modelmatch:
    print("Model Already in Registry:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        upload_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        parent_model = modelmatch[0].resource_name

else:
    print('This is a new model, creating in model registry')
    parent_model = ''

if upload_model:
    model = aiplatform.Model.upload(
        display_name = f'{SERIES}_{EXPERIMENT}',
        model_id = f'model_{SERIES}_{EXPERIMENT}',
        parent_model =  parent_model,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = "gs://my-project-media-campaign-cost/04/04/models/20231229122112/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
    )

This is a new model, creating in model registry
Creating Model
Create Model backing LRO: projects/683212519680/locations/australia-southeast1/models/model_04_04/operations/7024508210488803328
Model created. Resource name: projects/683212519680/locations/australia-southeast1/models/model_04_04@1
To use this Model in another session:
model = aiplatform.Model('projects/683212519680/locations/australia-southeast1/models/model_04_04@1')


In [223]:
expRun = aiplatform.ExperimentRun(run_name = RUN_NAME, experiment = EXPERIMENT_NAME)

expRun.log_params({
    'model.uri': model.uri,
    'model.display_name': model.display_name,
    'model.name': model.name,
    'model.resource_name': model.resource_name,
    'model.version_id': model.version_id,
    'model.versioned_resource_name': model.versioned_resource_name,
    'customJobs.display_name': customJob.display_name,
    'customJobs.resource_name': customJob.resource_name,
    'customJobs.link': job_link
})
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

# Review the experiment

In [224]:
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)
exp.get_data_frame()

,experiment_name,run_name,run_type,state,param.customJobs.display_name,param.experiment,param.customJobs.link,param.project_id,param.model.uri,param.customJobs.resource_name,param.data_source,param.model.display_name,param.model.versioned_resource_name,param.model.save,param.model.name,param.model.resource_name,param.test_msle,param.model.version_id,param.series,param.train_msle
0,experiment-04-04-sklearn-regression-ensemble,run-20231229122112,system.ExperimentRun,COMPLETE,04_04_20231229122112,04,https://console.cloud.google.com/vertex-ai/loc...,my-project-media-campaign-cost,gs://my-project-media-campaign-cost/04/04/mode...,projects/683212519680/locations/australia-sout...,bq://my-project-media-campaign-cost.media_camp...,04_04,projects/683212519680/locations/australia-sout...,/gcs/my-project-media-campaign-cost/04/04/mode...,model_04_04,projects/683212519680/locations/australia-sout...,0.092623,1,04,0.09135


# Retrieve or create the end point

In [204]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}",
        labels = {'series' : f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")

Creating Endpoint
Create Endpoint backing LRO: projects/683212519680/locations/australia-southeast1/endpoints/6048360687837642752/operations/4043072480611401728
Endpoint created. Resource name: projects/683212519680/locations/australia-southeast1/endpoints/6048360687837642752
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/683212519680/locations/australia-southeast1/endpoints/6048360687837642752')
Endpoint Created: projects/683212519680/locations/australia-southeast1/endpoints/6048360687837642752


In [206]:
model

resource name: projects/683212519680/locations/australia-southeast1/models/model_04_04

In [213]:
model.display_name

'04_04'

# Deploy the model

In [209]:
import xgboost 
import lightgbm
import catboost

In [ ]:
endpoint.deploy(
        model = model,
        deployed_model_display_name = model.display_name,
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )

In [ ]:
 endpoint.undeploy(deployed_model_id = deployed_model.id)